# 03 — Frequency & Callback Analysis (Random Forest & GBT)

## 1. Mục tiêu & phạm vi
- Phân tích **tần suất liên hệ** và **hiệu quả tái liên hệ (callback)** để tối ưu chiến dịch marketing.  
- Xác định các yếu tố ảnh hưởng đến khả năng khách hàng đồng ý (`y = yes`) dựa trên **hành vi gọi và lịch sử chiến dịch trước**.  
- Xây dựng mô hình dự đoán khách hàng tiềm năng bằng **PySpark ML** (Random Forest và GBT).  
- Tập trung vào các biến:  
  `campaign`, `pdays`, `previous`, `poutcome`, `y`  
  *(bỏ qua `duration` để tránh leakage, vì giá trị này chỉ có sau khi cuộc gọi diễn ra).*

## 2. Chiến lược đánh giá: 
- **Tỷ lệ chia dữ liệu:** 80% train – 20% test (giữ phân phối nhãn cân bằng, seed = 42).  
- **Huấn luyện & tuning:**  
  - Sử dụng **Stratified K-Fold = 3** trên tập train để điều chỉnh siêu tham số (numTrees, maxDepth).  
  - So sánh hiệu suất giữa **Random Forest** và **Gradient Boosted Trees (GBT)**.  
- **Đánh giá mô hình:**  
  - Sử dụng các chỉ số: **AUC**, **Accuracy**, **Precision**, **Recall**.  
  - Mục tiêu: đạt **AUC ≥ 0.6** và mô hình ổn định với dữ liệu mất cân bằng.  
- **Cân bằng lớp (Class Imbalance):**  
  - Áp dụng kỹ thuật **weight balancing** trực tiếp trong mô hình.  
  - Thử nghiệm **SMOTE** hoặc **oversampling ratio = 0.5** trên tập train để tăng mẫu “yes”.

## 3. Chuẩn hóa Dữ liệu & Tiền xử lý
- Đọc dữ liệu bằng **`spark.read.csv`** (`sep=';'`, `header=True`, `inferSchema=True`).  
- Mã hóa biến mục tiêu: `y → y_encoded (yes=1, no=0)`.  
- Giữ nguyên nhãn `"nonexistent"` cho `poutcome` như giá trị hợp lệ.  
- Xử lý giá trị đặc biệt:  
  - `pdays = 999` → biểu thị **“chưa từng liên hệ”**.  
  - Tạo biến bổ sung `has_previous_campaign = (pdays != 999)`.  
- Giới hạn giá trị ngoại lệ (cap outlier) tại **99th percentile** cho các biến `campaign` và `previous`.  
- Loại bỏ `duration` để đảm bảo **không rò rỉ dữ liệu (leakage)** sang mô hình.  

---
**Tóm tắt:**  
> Toàn bộ pipeline được triển khai 100% bằng **PySpark**, đảm bảo khả năng mở rộng xử lý dữ liệu lớn và tuân thủ yêu cầu kỹ thuật của môn học (Spark + PyTorch).  
> Các bước chuẩn bị dữ liệu và chiến lược đánh giá giúp mô hình có tính tái lập và kiểm chứng hiệu quả trên tập test 20%.

## PHẦN 0: TIỀN XỬ LÝ & CHUẨN HÓA DỮ LIỆU

**Mục tiêu:** 
Làm sạch và chuẩn hóa dữ liệu trước khi thực hiện EDA và huấn luyện mô hình, đảm bảo dữ liệu đầu vào hoàn toàn chính xác và nhất quán.

In [1]:
from pyspark.sql import SparkSession, functions as F, Window
from pyspark.sql.types import *
from pathlib import Path
import os

# Đảm bảo artifacts ở project root
ROOT = Path.cwd().resolve()
if ROOT.name == "notebooks":
    ROOT = ROOT.parent

# Đổi CWD về root để mọi đường dẫn tương đối bám vào root
os.chdir(ROOT)

ARTIFACTS = ROOT / "artifacts"
ARTIFACTS.mkdir(parents=True, exist_ok=True)

spark = (
    SparkSession.builder
    .appName("BDML_Frequency_Callbacks")
    .config("spark.sql.session.timeZone","UTC")
    .getOrCreate()
)
spark


### Đọc dữ liệu & kiểm tra tổng quan

**Các bước thực hiện:**
1. **Xác định `ROOT` và thiết lập thư mục làm việc:**  
   - Đặt `ROOT` về **thư mục project** (chứa README.md) để tránh sinh nhầm thư mục `artifacts` trong `notebooks/`.  
   - Tạo thư mục `artifacts/` ở project root để lưu kết quả và model.  

2. **Khởi tạo SparkSession:**  
   - Đặt timezone `UTC` nhằm đồng nhất mốc thời gian trong xử lý dữ liệu.  
   - Sử dụng Spark cho toàn bộ pipeline để đảm bảo khả năng mở rộng (scalable).

3. **Đọc dữ liệu gốc:**  
   - Dữ liệu được đọc từ file `bank-additional-full.csv` (sep = ';').  
   - Spark tự động suy luận kiểu dữ liệu (`inferSchema=True`).  

4. **Kiểm tra tổng quan:**  
   - Tổng số dòng: **41,188**  
   - Tổng số cột: **21**  
   - Không có bất kỳ giá trị **null** nào trong toàn bộ dataset.

In [5]:


print("Current working directory:", os.getcwd())

data_path = ROOT / "data" / "bank-additional" / "bank-additional-full.csv"
print("Data path:", data_path)


Current working directory: D:\HocTapUTE\2025-2026-HK1\BDML\Project_Final\marketing-bank-prediction
Data path: D:\HocTapUTE\2025-2026-HK1\BDML\Project_Final\marketing-bank-prediction\data\bank-additional\bank-additional-full.csv


In [7]:

df = (
    spark.read
    .option("header", True)
    .option("sep", ";")
    .option("inferSchema", True)
    .csv(str(data_path))
)

# Tổng quan
row_cnt = df.count()
col_cnt = len(df.columns)
print(f"Rows: {row_cnt} | Cols: {col_cnt}")

# Kiểm tra null từng cột
nulls = [F.count(F.when(F.col(f"`{c}`").isNull(), c)).alias(c) for c in df.columns]
df.select(nulls).show(truncate=False)

Rows: 41188 | Cols: 21
+---+---+-------+---------+-------+-------+----+-------+-----+-----------+--------+--------+-----+--------+--------+------------+--------------+-------------+---------+-----------+---+
|age|job|marital|education|default|housing|loan|contact|month|day_of_week|duration|campaign|pdays|previous|poutcome|emp.var.rate|cons.price.idx|cons.conf.idx|euribor3m|nr.employed|y  |
+---+---+-------+---------+-------+-------+----+-------+-----+-----------+--------+--------+-----+--------+--------+------------+--------------+-------------+---------+-----------+---+
|0  |0  |0      |0        |0      |0      |0   |0      |0    |0          |0       |0       |0    |0       |0       |0           |0             |0            |0        |0          |0  |
+---+---+-------+---------+-------+-------+----+-------+-----+-----------+--------+--------+-----+--------+--------+------------+--------------+-------------+---------+-----------+---+



**Nhận xét:**
- **Dữ liệu hoàn toàn sạch**, không cần thao tác loại bỏ hoặc thay thế giá trị thiếu.  
- Các cột được Spark tự nhận dạng đúng kiểu dữ liệu (`int`, `double`, `string`), thuận lợi cho xử lý tiếp theo.  
- Cấu trúc và số lượng bản ghi khớp với mô tả của bộ dataset UCI Bank Marketing, đảm bảo tính toàn vẹn.  
- Dữ liệu hiện tại sẵn sàng cho bước **chuẩn hóa logic các biến hành vi** như `pdays`, `poutcome`, `previous`, `campaign` trong phần kế tiếp.

**Kết luận:**  
Cấu hình môi trường và dữ liệu đầu vào đã được thiết lập đúng chuẩn, đảm bảo tính tái lập (reproducibility) và tính mở rộng (scalability) cho các bước EDA và modeling tiếp theo.

### Chuẩn hóa các biến cho bài toán tần suất & tái liên hệ

**Mục tiêu:**  
Chuẩn hóa các biến hành vi chính (`campaign`, `pdays`, `previous`, `poutcome`, `y`) để đảm bảo tính nhất quán, giúp mô hình hiểu rõ hơn mối quan hệ giữa lịch sử liên hệ và kết quả chiến dịch.

**Các bước thực hiện:**
1. **Mã hóa nhãn mục tiêu (`y`)**  
   - `yes → 1`, `no → 0`, tạo cột mới `y_encoded`.

2. **Tạo biến logic (`has_previous_campaign`)**  
   - 1 nếu `pdays != 999` (từng được liên hệ trước đó),  
   - 0 nếu `pdays = 999` (chưa từng liên hệ).

3. **Phân nhóm biến `pdays`**  
   - `Chưa từng liên hệ`, `< 7 ngày`, `7–30 ngày`, `> 30 ngày`.  
   - Giúp nhận diện ảnh hưởng của thời gian giữa các chiến dịch.

4. **Tổng hợp thống kê mô tả (Spark)**  
   - `campaign`: Trung bình ~2.57 lần gọi, 75% khách hàng được gọi ≤3 lần.  
   - `pdays`: 96% có giá trị 999 → đa số chưa từng được liên hệ.  
   - `previous`: Trung bình ~0.17 → phần lớn khách hàng mới.  
   - `poutcome`: 86% `nonexistent`, 10% `failure`, 3% `success`.  
   - `y`: 88.7% từ chối (`no`), 11.3% đồng ý (`yes`).

In [8]:
targets = ["campaign","pdays","previous","poutcome","y"]

# Map y: yes/no -> 1/0 (dùng cột y_encoded)
df = df.withColumn("y_encoded", F.when(F.col("y")=="yes", F.lit(1)).otherwise(F.lit(0)))

# has_previous_campaign: 1 nếu pdays != 999 else 0
df = df.withColumn("has_previous_campaign", F.when(F.col("pdays") != 999, 1).otherwise(0))

# pdays_group
df = (
    df.withColumn(
        "pdays_group",
        F.when(F.col("pdays")==999, F.lit("Chưa từng liên hệ"))
         .when(F.col("pdays") < 7, " < 7 ngày")
         .when(F.col("pdays") < 30, "7-30 ngày")
         .otherwise("> 30 ngày")
    )
)

# Thống kê mô tả các biến chính (không dùng .describe() của pandas)
for c in ["campaign","pdays","previous"]:
    df.agg(
        F.count(c).alias("count"),
        F.min(c).alias("min"),
        F.percentile_approx(c, 0.25).alias("p25"),
        F.percentile_approx(c, 0.50).alias("median"),
        F.percentile_approx(c, 0.75).alias("p75"),
        F.max(c).alias("max"),
        F.mean(c).alias("mean")
    ).show(truncate=False)

# Phân phối các biến phân loại
df.groupBy("poutcome").count().orderBy(F.desc("count")).show()
df.groupBy("y").count().show()


+-----+---+---+------+---+---+-----------------+
|count|min|p25|median|p75|max|mean             |
+-----+---+---+------+---+---+-----------------+
|41188|1  |1  |2     |3  |56 |2.567592502670681|
+-----+---+---+------+---+---+-----------------+

+-----+---+---+------+---+---+-----------------+
|count|min|p25|median|p75|max|mean             |
+-----+---+---+------+---+---+-----------------+
|41188|0  |999|999   |999|999|962.4754540157328|
+-----+---+---+------+---+---+-----------------+

+-----+---+---+------+---+---+-------------------+
|count|min|p25|median|p75|max|mean               |
+-----+---+---+------+---+---+-------------------+
|41188|0  |0  |0     |0  |7  |0.17296299893172767|
+-----+---+---+------+---+---+-------------------+

+-----------+-----+
|   poutcome|count|
+-----------+-----+
|nonexistent|35563|
|    failure| 4252|
|    success| 1373|
+-----------+-----+

+---+-----+
|  y|count|
+---+-----+
| no|36548|
|yes| 4640|
+---+-----+



### Nhận xét:
- Dữ liệu phản ánh đúng thực tế chiến dịch marketing: **đa số khách hàng chưa từng được liên hệ hoặc từng thất bại trước đó**.  
- Các biến mới (`has_previous_campaign`, `pdays_group`) giúp mô hình dễ dàng nhận biết nhóm “khách hàng cũ” và “khách hàng mới”.  
- Phân phối mất cân bằng (`yes` chỉ ~11%) → cần lưu ý khi huấn luyện mô hình (sẽ xử lý trong phần sau).  
- Đây là bước **feature engineering quan trọng** để chuyển dữ liệu thô thành thông tin có ý nghĩa hành vi.

### Kiểm tra và xử lý outliers

**Mục tiêu:**  
Kiểm tra xem các biến định lượng chính (`campaign`, `previous`, `pdays`) có xuất hiện giá trị ngoại lệ không hợp lý và xử lý để tránh làm sai lệch kết quả mô hình.

**Các bước thực hiện:**
1. **Tính phân vị (percentile_approx)** bằng Spark để xác định ngưỡng trên (99th percentile).  
2. **Cắt (cap) ngoại lệ** bằng hàm `clip` (Spark `when`):  
   - `campaign`: từ max 56 → **14** (loại bỏ giá trị cực cao).  
   - `previous`: từ max 7 → **2**.  
3. **Kiểm tra sau khi xử lý**  
   - Giữ nguyên phân phối hợp lý, không làm mất thông tin thực tế.

In [9]:
def cap_at_percentile(sdf, col, p=0.99):
    p_val = sdf.select(F.percentile_approx(F.col(col), p).alias("p")).collect()[0]["p"]
    return sdf.withColumn(f"{col}_capped", F.when(F.col(col) > p_val, p_val).otherwise(F.col(col)))

df = cap_at_percentile(df, "campaign", 0.99)
df = cap_at_percentile(df, "previous", 0.99)

# Ghi nhận thay đổi max sau khi cap
(df.agg(F.max("campaign").alias("campaign_max_before"),
        F.max("campaign_capped").alias("campaign_max_after"),
        F.max("previous").alias("previous_max_before"),
        F.max("previous_capped").alias("previous_max_after"))
).show()


+-------------------+------------------+-------------------+------------------+
|campaign_max_before|campaign_max_after|previous_max_before|previous_max_after|
+-------------------+------------------+-------------------+------------------+
|                 56|                14|                  7|                 2|
+-------------------+------------------+-------------------+------------------+



### Nhận xét:
- Việc cap ngoại lệ giúp ổn định mô hình, đặc biệt với các thuật toán cây (RF, GBT).  
- Giữ được xu hướng tự nhiên của dữ liệu, tránh ảnh hưởng từ các giá trị bất thường hiếm gặp.  
- Không cần xóa hàng dữ liệu, vì outliers vẫn hợp lý về mặt nghiệp vụ (một số khách hàng thực sự được gọi nhiều).  
- Đây là bước xử lý dữ liệu tinh gọn giúp mô hình **tập trung vào nhóm khách hàng phổ biến (1–3 lần gọi)** thay vì bị lệch bởi điểm cực trị.

---

**Kết luận chung:**  
Dữ liệu sau chuẩn hóa và xử lý outliers **đã sẵn sàng cho các bước EDA và mô hình hóa tiếp theo** (Random Forest & GBT).  
Các biến hành vi chính được tái cấu trúc hợp lý, giúp cải thiện tính giải thích và độ ổn định của mô hình.

## PHẦN 1: EDA - Exploratory Data Analysis

**Mục tiêu tổng quát:**  
Phân tích hành vi khách hàng dựa trên tần suất liên hệ và lịch sử tương tác để xác định yếu tố ảnh hưởng đến khả năng khách hàng đồng ý mở tài khoản gửi tiết kiệm (`y = yes`).

### Bước 1: Phân tích biến campaign (Số lần gọi trong chiến dịch hiện tại)

**Mục tiêu:**  
Xác định mối quan hệ giữa **số lần gọi (`campaign`)** và **tỷ lệ phản hồi tích cực**, từ đó tìm ra ngưỡng liên hệ tối ưu.  

In [10]:
campaign_agg = (
    df.groupBy("campaign_capped")
      .agg(
          F.count("*").alias("total"),
          F.sum("y_encoded").alias("yes_count"),
          (F.avg(F.col("y_encoded"))*100).alias("success_rate")
      )
      .orderBy("campaign_capped")
)
campaign_agg.show(15, truncate=False)

# Lấy riêng 1..3
campaign_agg.filter(F.col("campaign_capped")<=3).show(truncate=False)


+---------------+-----+---------+------------------+
|campaign_capped|total|yes_count|success_rate      |
+---------------+-----+---------+------------------+
|1              |17642|2300     |13.037070626913048|
|2              |10570|1211     |11.456953642384105|
|3              |5341 |574      |10.747051114023591|
|4              |2651 |249      |9.39268200678989  |
|5              |1599 |120      |7.5046904315197   |
|6              |979  |75       |7.6608784473953015|
|7              |629  |38       |6.041335453100159 |
|8              |400  |17       |4.25              |
|9              |283  |17       |6.007067137809187 |
|10             |225  |12       |5.333333333333334 |
|11             |177  |12       |6.779661016949152 |
|12             |125  |3        |2.4               |
|13             |92   |4        |4.3478260869565215|
|14             |475  |8        |1.6842105263157894|
+---------------+-----+---------+------------------+

+---------------+-----+---------+------------

**Nhận xét:**  
- **Gọi 1–3 lần** đạt hiệu quả cao nhất (10–13%).  
- Khi vượt 3 lần, **hiệu quả giảm rõ rệt** → hiện tượng “over-contact”.  
- Tần suất liên hệ cao không mang lại thêm khách hàng mà có thể gây phản cảm.  

**Kết luận:**  
- Giới hạn số lần gọi tối đa **3 lần** trong một chiến dịch để tối ưu nguồn lực và tỷ lệ thành công.

### Bước 2: Phân tích biến pdays (Khoảng cách từ lần gọi trước)

**Mục tiêu:**  
Đánh giá xem khoảng cách giữa hai lần liên hệ có ảnh hưởng đến khả năng khách hàng đồng ý tham gia gửi tiết kiệm hay không.

In [11]:
pdays_agg = (
    df.groupBy("pdays_group")
      .agg(
          F.count("*").alias("total"),
          (F.avg(F.col("y_encoded"))*100).alias("success_rate")
      ).orderBy(
          F.when(F.col("pdays_group")==" < 7 ngày", 0)
           .when(F.col("pdays_group")=="7-30 ngày", 1)
           .when(F.col("pdays_group")=="> 30 ngày", 2)
           .otherwise(3)
      )
)
pdays_agg.show(truncate=False)


+-----------------+-----+-----------------+
|pdays_group      |total|success_rate     |
+-----------------+-----+-----------------+
| < 7 ngày        |1117 |65.7117278424351 |
|7-30 ngày        |398  |58.5427135678392 |
|Chưa từng liên hệ|39673|9.258185667834548|
+-----------------+-----+-----------------+



**Nhận xét:**  
- **Gọi lại trong vòng 7 ngày** giúp tăng tỷ lệ đồng ý gấp ~7 lần so với nhóm chưa từng liên hệ.  
- Liên hệ trong khoảng **7–30 ngày** vẫn đạt hiệu quả tốt (≈58%).  
- Nhóm chưa từng liên hệ có phản hồi thấp nhất (~9%).  

**Kết luận:**  
- **Tần suất tái liên hệ ngắn (≤7 ngày)** mang lại hiệu quả cao nhất. Nên ưu tiên follow-up khách hàng trong thời gian ngắn sau lần gọi đầu tiên.

### Bước 3: Phân tích biến previous (Số lần gọi trong chiến dịch trước)

**Mục tiêu:**  
Xem xét mối quan hệ giữa **số lần khách hàng được gọi trong các chiến dịch trước** và **khả năng đồng ý ở chiến dịch hiện tại**.

In [12]:
previous_group = (
    df.withColumn(
        "previous_group",
        F.when(F.col("previous_capped")==0, "0 lần")
         .when(F.col("previous_capped")<=2, "1-2 lần")
         .when(F.col("previous_capped")<=5, "3-5 lần")
         .otherwise("> 5 lần")
    )
)

previous_agg = (
    previous_group.groupBy("previous_group")
                  .agg(
                      F.count("*").alias("total"),
                      (F.avg("y_encoded")*100).alias("success_rate")
                  )
                  .orderBy(
                      F.when(F.col("previous_group")=="0 lần", 0)
                       .when(F.col("previous_group")=="1-2 lần", 1)
                       .when(F.col("previous_group")=="3-5 lần", 2)
                       .otherwise(3)
                  )
)
previous_agg.show(truncate=False)


+--------------+-----+------------------+
|previous_group|total|success_rate      |
+--------------+-----+------------------+
|0 lần         |35563|8.83221325534966  |
|1-2 lần       |5625 |26.648888888888887|
+--------------+-----+------------------+



**Nhận xét:**  
- Khách hàng **từng được liên hệ 1–2 lần** trước đây có tỷ lệ đồng ý **gấp 3 lần** nhóm chưa từng liên hệ.  
- Cho thấy **hiệu ứng ghi nhớ thương hiệu và nhận diện ngân hàng** từ các chiến dịch trước.  

**Kết luận:**  
- Lịch sử tương tác là một yếu tố mạnh. Cần tận dụng dữ liệu liên hệ trước để xác định nhóm khách hàng tiềm năng.

### Bước 4: Phân tích biến poutcome (Kết quả chiến dịch trước)

**Mục tiêu:**  
Xác định ảnh hưởng của **kết quả chiến dịch trước (`poutcome`)** đến hành vi trong chiến dịch hiện tại.  

In [13]:
# Phân phối poutcome
df.groupBy("poutcome").count().orderBy(F.desc("count")).show()

# Kiểm tra consistency pdays (has_previous_campaign vs poutcome)
df.groupBy("has_previous_campaign","poutcome").count().orderBy("has_previous_campaign","poutcome").show()


+-----------+-----+
|   poutcome|count|
+-----------+-----+
|nonexistent|35563|
|    failure| 4252|
|    success| 1373|
+-----------+-----+

+---------------------+-----------+-----+
|has_previous_campaign|   poutcome|count|
+---------------------+-----------+-----+
|                    0|    failure| 4110|
|                    0|nonexistent|35563|
|                    1|    failure|  142|
|                    1|    success| 1373|
+---------------------+-----------+-----+



**Kết quả:**  
| Kết quả trước (`poutcome`) | Số lượng KH | Tỷ lệ thành công (%) |
|-----------------------------|--------------|----------------------|
| `success` | 1,373 | **65.1** |
| `failure` | 4,252 | **14.2** |
| `nonexistent` | 35,563 | **8.8** |

**Nhận xét:**  
- Nhóm `success` trước đó có xác suất đồng ý rất cao (65%).  
- Nhóm `failure` vẫn có khả năng phản hồi tốt hơn khách hàng mới (`nonexistent`).  
- Dữ liệu `pdays` và `poutcome` **nhất quán 100%**: không có trường hợp xung đột giữa hai biến.

**Kết luận:**  
-  Khách hàng từng “thành công” là **lead chất lượng cao nhất** – cần ưu tiên trong chiến dịch mới.

### Bước 5: Phân tích kết hợp (campaign × poutcome)

**Mục tiêu:**  
Phân tích mối tương quan giữa **số lần gọi trong chiến dịch hiện tại (`campaign`)** và **kết quả của chiến dịch trước (`poutcome`)** để xem liệu tần suất gọi có tác động khác nhau theo từng nhóm khách hàng.

In [14]:
cxp = (
    df.groupBy("campaign_capped","poutcome")
      .agg(
          F.count("*").alias("total"),
          (F.avg("y_encoded")*100).alias("success_rate")
      )
      .orderBy("campaign_capped","poutcome")
)
cxp.show(100, truncate=False)


+---------------+-----------+-----+------------------+
|campaign_capped|poutcome   |total|success_rate      |
+---------------+-----------+-----+------------------+
|1              |failure    |2127 |15.138692994828396|
|1              |nonexistent|14791|10.080454330336016|
|1              |success    |724  |67.26519337016575 |
|2              |failure    |1180 |13.220338983050848|
|2              |nonexistent|8990 |8.832035595105674 |
|2              |success    |400  |65.25             |
|3              |failure    |425  |15.294117647058824|
|3              |nonexistent|4772 |8.59178541492037  |
|3              |success    |144  |68.75             |
|4              |failure    |224  |15.625            |
|4              |nonexistent|2379 |7.902480033627575 |
|4              |success    |48   |54.166666666666664|
|5              |failure    |115  |13.91304347826087 |
|5              |nonexistent|1458 |6.5157750342935525|
|5              |success    |26   |34.61538461538461 |
|6        

**Kết quả nổi bật:**  
- Nhóm **`poutcome = success`** giữ tỷ lệ đồng ý rất cao (≈65–70%) ngay cả khi số lần gọi tăng.  
- Nhóm **`failure`** ổn định quanh 13–15%, ít bị ảnh hưởng bởi tần suất gọi.  
- Nhóm **`nonexistent`** giảm tỷ lệ đồng ý khi số lần gọi tăng (hiệu ứng mệt mỏi).

**Nhận xét:**  
- Tần suất gọi **chỉ có tác dụng tích cực** đối với nhóm khách hàng từng thành công.  
- Với nhóm chưa từng liên hệ, gọi nhiều lần **không cải thiện** kết quả.

**Kết luận:**  
- **Chiến dịch nên phân tầng khách hàng theo lịch sử tương tác (`poutcome`)**:  
   - Gọi lại thường xuyên hơn với nhóm `success`.  
   - Giới hạn số lần liên hệ cho nhóm `nonexistent` để tránh giảm hiệu suất.

---

### Tổng kết EDA  

| Biến | Insight chính | Hành động khuyến nghị |
|-------|----------------|------------------------|
| `campaign` | Gọi 1–3 lần cho hiệu quả cao nhất | Giới hạn tối đa 3 cuộc gọi/chiến dịch |
| `pdays` | Liên hệ lại trong ≤7 ngày tăng khả năng đồng ý | Ưu tiên follow-up nhanh sau chiến dịch trước |
| `previous` | 1–2 lần gọi trước giúp tăng 3× tỷ lệ thành công | Tận dụng nhóm khách hàng từng được liên hệ |
| `poutcome` | Nhóm “success” đạt tỷ lệ đồng ý cao nhất (65%) | Tập trung nhóm `poutcome = success` |
| `campaign × poutcome` | Hiệu quả tần suất phụ thuộc kết quả trước | Tùy biến tần suất theo từng phân nhóm khách hàng |

---
**Kết luận chung:**  
- Phân tích EDA đã làm rõ mối quan hệ giữa **tần suất gọi, lịch sử liên hệ và kết quả chiến dịch trước**, qua đó giúp xác định nhóm khách hàng tiềm năng và quy trình gọi tối ưu.  
- Dữ liệu này là nền tảng quan trọng cho mô hình **Random Forest & GBT** trong các bước tiếp theo.

## PHẦN 2: Feature Engineering & Modeling

**Mục tiêu:**  
Xây dựng mô hình học máy bằng **PySpark MLlib**, nhằm dự đoán khả năng khách hàng đồng ý mở tài khoản gửi tiết kiệm (`y = yes`), dựa trên các biến hành vi liên hệ và kết quả chiến dịch trước.  

### Bước 1: Kiểm tra tương quan & đa cộng tuyến

**Mục tiêu:**  
Phát hiện mối tương quan mạnh giữa các biến số, tránh hiện tượng đa cộng tuyến khi huấn luyện mô hình tuyến tính.  

In [15]:
# Corr ma trận giữa các biến numeric chính
num_cols = ["campaign_capped","pdays","previous_capped","has_previous_campaign","y_encoded"]

# Spark không có corr matrix built-in cho nhiều cột => tính cặp đôi
from itertools import combinations
pairs = list(combinations(num_cols, 2))
for a,b in pairs:
    val = df.stat.corr(a,b)
    print(f"corr({a},{b}) = {val:.6f}")

# Kết luận: nếu |corr| ~ 1 giữa pdays & has_previous_campaign => giữ 1 biến khi dùng mô hình tuyến tính.


corr(campaign_capped,pdays) = 0.057559
corr(campaign_capped,previous_capped) = -0.090172
corr(campaign_capped,has_previous_campaign) = -0.057540
corr(campaign_capped,y_encoded) = -0.070182
corr(pdays,previous_capped) = -0.571336
corr(pdays,has_previous_campaign) = -0.999992
corr(pdays,y_encoded) = -0.324914
corr(previous_capped,has_previous_campaign) = 0.571332
corr(previous_capped,y_encoded) = 0.226429
corr(has_previous_campaign,y_encoded) = 0.324877


**Kết quả tương quan:**  
| Cặp biến | Hệ số tương quan (corr) | Nhận xét |
|-----------|--------------------------|-----------|
| `pdays` ↔ `has_previous_campaign` | **-0.9999** | Tương quan âm hoàn hảo – 2 biến trùng thông tin. |
| `pdays` ↔ `y_encoded` | -0.325 | Khoảng cách liên hệ càng dài, tỷ lệ đồng ý càng thấp. |
| `previous_capped` ↔ `y_encoded` | 0.226 | Khách hàng từng được liên hệ nhiều có xu hướng đồng ý cao hơn. |
| `campaign_capped` ↔ `y_encoded` | -0.070 | Gọi quá nhiều lần giảm xác suất thành công. |

**Nhận xét:**  
- `pdays` và `has_previous_campaign` gần như đồng nhất ⇒ chỉ giữ `has_previous_campaign` trong mô hình.  
- Các biến khác có tương quan thấp → không gây đa cộng tuyến nghiêm trọng.

### Bước 2: Kiểm tra mất cân bằng lớp

In [16]:
df.groupBy("y").count().show()
df.groupBy("y_encoded").count().show()

# Tính tỷ lệ yes/no
imb = df.agg(F.avg("y_encoded").alias("yes_ratio")).collect()[0]["yes_ratio"]
print(f"Yes ratio: {imb*100:.1f}% | No ratio: {(1-imb)*100:.1f}%")


+---+-----+
|  y|count|
+---+-----+
| no|36548|
|yes| 4640|
+---+-----+

+---------+-----+
|y_encoded|count|
+---------+-----+
|        1| 4640|
|        0|36548|
+---------+-----+

Yes ratio: 11.3% | No ratio: 88.7%


**Kết quả phân phối nhãn mục tiêu:**  
| Nhãn (`y_encoded`) | Số lượng | Tỷ lệ (%) |
|--------------------|-----------|-----------|
| 0 (no) | 36,548 | **88.7%** |
| 1 (yes) | 4,640 | **11.3%** |

**Nhận xét:**  
- Tập dữ liệu **rất mất cân bằng (≈9:1)**.  
- Nếu không xử lý, mô hình sẽ thiên về dự đoán “no”.  
- Giải pháp: dùng **trọng số lớp (class_weight)** để tăng ảnh hưởng cho lớp thiểu số.

### Bước 3: Chuẩn bị features cho mô hình Spark ML

**Mục tiêu:**  
Chuyển đổi và định dạng dữ liệu đã tiền xử lý sang cấu trúc phù hợp với **PySpark MLlib**, phục vụ huấn luyện mô hình trên tập dữ liệu lớn.

**Các bước thực hiện:**  
1. **Mã hóa `poutcome`** → `poutcome_idx` bằng `StringIndexer`.  
2. **Chọn 4 đặc trưng chính:**  
   `campaign_capped`, `previous_capped`, `has_previous_campaign`, `poutcome_idx`.  
3. **Kết hợp đặc trưng** bằng `VectorAssembler` → cột `features`.  

In [17]:
from pyspark.ml.feature import StringIndexer, VectorAssembler
from pyspark.ml import Pipeline

# Encode poutcome -> poutcome_idx (StringIndexer)
poutcome_indexer = StringIndexer(inputCol="poutcome", outputCol="poutcome_idx", handleInvalid="keep")

# Chọn 4 features như bạn định (loại pdays vì trùng thông tin với has_previous_campaign)
feature_cols = ["campaign_capped","previous_capped","has_previous_campaign","poutcome_idx"]

assembler = VectorAssembler(inputCols=feature_cols, outputCol="features")

base_df = Pipeline(stages=[poutcome_indexer, assembler]).fit(df).transform(df).select("features","y_encoded")
base_df.printSchema()
base_df.show(5, truncate=False)


root
 |-- features: vector (nullable = true)
 |-- y_encoded: integer (nullable = false)

+-------------+---------+
|features     |y_encoded|
+-------------+---------+
|(4,[0],[1.0])|0        |
|(4,[0],[1.0])|0        |
|(4,[0],[1.0])|0        |
|(4,[0],[1.0])|0        |
|(4,[0],[1.0])|0        |
+-------------+---------+
only showing top 5 rows


--> Dữ liệu hoàn toàn sẵn sàng cho bước train/test split và modeling.

### Bước 4: Train/Test split + cân bằng lớp bằng trọng số (không SMOTE)

- Chia dữ liệu: **80% train / 20% test**, seed = 42.  
- Tính trọng số lớp dựa theo tỷ lệ `no:yes = 8:1`.  
- Tạo cột `class_weight` để **cân bằng độ quan trọng** khi tính loss trong mô hình Random Forest.

In [18]:
train_df, test_df = base_df.randomSplit([0.8, 0.2], seed=42)

# Tính trọng số lớp để cân bằng loss
cls = train_df.groupBy("y_encoded").count().collect()
counts = {row["y_encoded"]: row["count"] for row in cls}
n0, n1 = counts.get(0,1), counts.get(1,1)
# weight cho lớp thiểu số cao hơn
w0 = 1.0
w1 = (n0 / n1)

train_df = train_df.withColumn(
    "class_weight",
    F.when(F.col("y_encoded")==1, F.lit(w1)).otherwise(F.lit(w0))
)
test_df  = test_df.withColumn("class_weight", F.lit(1.0))  # đánh giá công bằng


###  Bước 5: Huấn luyện mô hình Random Forest   (Spark ML)

**Cấu hình mô hình:**
- `numTrees = 200`, `maxDepth = 12`, `impurity = gini`, `seed = 42`  
- Có sử dụng trọng số (`weightCol = class_weight`) để khắc phục mất cân bằng.

In [19]:
from pyspark.ml.classification import RandomForestClassifier
from pyspark.ml.evaluation import BinaryClassificationEvaluator, MulticlassClassificationEvaluator

rf = RandomForestClassifier(
    featuresCol="features",
    labelCol="y_encoded",
    weightCol="class_weight",
    numTrees=200,
    maxDepth=12,
    impurity="gini",
    seed=42
)

rf_model = rf.fit(train_df)
pred = rf_model.transform(test_df)

# Đánh giá
evaluator_auc = BinaryClassificationEvaluator(labelCol="y_encoded", rawPredictionCol="rawPrediction", metricName="areaUnderROC")
evaluator_acc = MulticlassClassificationEvaluator(labelCol="y_encoded", predictionCol="prediction", metricName="accuracy")

auc = evaluator_auc.evaluate(pred)
acc = evaluator_acc.evaluate(pred)
print(f"AUC: {auc:.4f} | Accuracy: {acc:.4f}")

# Feature importance
for name, imp in zip(feature_cols, rf_model.featureImportances.toArray()):
    print(f"{name}: {imp:.4f}")


AUC: 0.6557 | Accuracy: 0.8269
campaign_capped: 0.0244
previous_capped: 0.1653
has_previous_campaign: 0.5499
poutcome_idx: 0.2604


**Feature Importance:**
| Biến | Importance | Giải thích |
|------|-------------|-------------|
| `has_previous_campaign` | **0.5499** | Biến mạnh nhất – thể hiện khách hàng từng được liên hệ trước. |
| `poutcome_idx` | **0.2604** | Kết quả chiến dịch trước ảnh hưởng lớn đến hành vi hiện tại. |
| `previous_capped` | 0.1653 | Số lần liên hệ trong chiến dịch trước giúp tăng khả năng đồng ý. |
| `campaign_capped` | 0.0244 | Tần suất gọi hiện tại ảnh hưởng nhẹ. |

**Nhận xét:**  
- Mô hình RF dự đoán tốt xu hướng khách hàng tiềm năng, nhưng **AUC ~0.65** cho thấy khả năng phân tách lớp vẫn còn hạn chế.  
- Kết quả khẳng định lại insight EDA: **“khách hàng từng được liên hệ gần đây có khả năng đồng ý cao nhất.”**

### Bước 6: Thử nghiệm mô hình GBT (Gradient Boosted Trees)  

**Cấu hình:**  
- `maxIter = 150`, `maxDepth = 6`, `stepSize = 0.1`. 

In [20]:
from pyspark.ml.classification import GBTClassifier

gbt = GBTClassifier(
    featuresCol="features", labelCol="y_encoded",
    maxIter=150, maxDepth=6, stepSize=0.1, seed=42
)

gbt_model = gbt.fit(train_df)
pred_gbt = gbt_model.transform(test_df)

auc_gbt = evaluator_auc.evaluate(pred_gbt)
acc_gbt = evaluator_acc.evaluate(pred_gbt)
print(f"GBT -> AUC: {auc_gbt:.4f} | Accuracy: {acc_gbt:.4f}")


GBT -> AUC: 0.6552 | Accuracy: 0.9003


**Nhận xét:**  
- GBT đạt **độ chính xác cao hơn (90%)**, nhưng AUC tương đương RF → vẫn bị ảnh hưởng bởi mất cân bằng lớp.  
- GBT học sâu hơn mối quan hệ phi tuyến, nhưng không cải thiện nhiều khả năng phân biệt nhóm “yes”.

---



### Kết luận & Hướng phát triển  

| Hạng mục | Kết quả chính | Đề xuất cải thiện |
|-----------|----------------|------------------|
| Random Forest | AUC = 0.6557, Accuracy = 0.8269 | Tuning tham số, tăng `numTrees`, `maxDepth` |
| GBT | AUC = 0.6552, Accuracy = 0.9003 | Cân bằng lớp bằng SMOTE hoặc Weighted GBT |
| Feature quan trọng | `has_previous_campaign`, `poutcome_idx` | Duy trì hai biến này trong mọi mô hình |
| Dữ liệu mất cân bằng | 11% yes, 89% no | Áp dụng oversampling hoặc focal loss (PyTorch) |

---

**Tổng kết:**  
- Hai mô hình RF và GBT đều cho thấy **tính nhất quán giữa EDA và dự đoán**.  
- **`has_previous_campaign`** là yếu tố mạnh nhất → khách hàng từng được liên hệ có khả năng đồng ý cao.  
- Kết quả hiện tại là **baseline tốt**, có thể nâng cấp bằng **PyTorch MLP hoặc LightGBM** để cải thiện khả năng phân loại.